# How does bee presence observations correlate with latitude throughout the year? An exploration of linear regression

This code uses a pre-written package called records to pull occurance data from GBIF. In this case we tested it using Bombus (bees). A linear regression was written to test the relationship between latitude at which bees are found vs month of the year.

In [3]:
#should replace with importing the package
#!/usr/bin/env python

"a package for pulling occurrence data from GBIF"


import requests
import pandas as pd
import numpy as np


class Records:
    """
    Returns a Records class instance with GBIF occurrence records stored
    in a pandas DataFrame for a queried taxon between a range of years.
    Parameters:
    -----------
    q: str
        Query taxonomic name.
    interval: tuple
        Range of years to return results for. Should be (min, max) tuple.
    Attributes:
    -----------
    baseurl: The REST API URL for GBIF.org.
    params: The parameter dictionary to filter GBIF search.
    df: Pandas DataFrame with returned records.
    sdf: A view of the 'df' DataFrame selecting only three relevant columns.
    """
    def __init__(self, q, interval, **kwargs):
        # the API url for searching GBIF occurrences
        self.baseurl = "http://api.gbif.org/v1/occurrence/search?"

        # the default REST API options plus user entered args
        self.params = {
            'q': q,
            'year': ",".join([str(i) for i in interval]),
            'basisOfRecord': "PRESERVED_SPECIMEN",
            'hasCoordinate': "true",
            'hasGeospatialIssue': "false",
            "country": "US",
            "offset": "0",
            "limit": "300",
        }

        # allow users to enter or modify other params using kwargs
        self.params.update(kwargs)

        # run the request query until all records are obtained
        self.df = pd.DataFrame(self._get_all_records())

    @property
    def sdf(self):
        """
        Return a copy of the current .df dataframe selecting only the three
        most generally relevant columns: species, year, and stateProvince.
        This is only meant for viewing and will raise a warning if you try to
        modify it since it is a copy, and thus you would be setting values on
        a selection of a selection. See pandas docs in the warning for detalis.
        """
        return self.df[["species", "year", "country", "stateProvince"]]

    def _get_all_records(self):
        "iterate until end of records"
        data = []
        while 1:
            # make request and store results
            res = requests.get(
                url=self.baseurl,
                params=self.params,
            )

            # check for errors
            res.raise_for_status()

            # increment counter
            self.params["offset"] = str(int(self.params["offset"]) + 300)

            # get data as json list of dicts and add to 'data' list
            idata = res.json()
            data += idata["results"]

            # stop when end of record is reached
            if idata["endOfRecords"]:
                break

        return data


class Epochs:
    """
    Returns an Epochs class instance that includes GBIF occurrence records
    and stores an extra label with each query that includes the interval
    (epoch) during which those records were collected, and returns all
    records in a sorted pandas dataframe.
    Parameters:
    -----------
    q: str
        Query taxonomic name.
    start: int
        Earliest year from which to search for records.
    end: int
        Latest year from which to search for records.
    epochsize: int
        of years to return results for. Should be (min, max) tuple.
    Attributes:
    -----------
    df: Pandas DataFrame with returned records.
    sdf: A view of the 'df' DataFrame selecting only four relevant columns.
    """
    def __init__(self, q, start, end, epochsize, **kwargs):

        # make range of epochs
        epochs = range(start, end, epochsize)

        # get Record objects across the epoch range
        rdicts = {
            i: Records(q, (i, i + epochsize), **kwargs) for i in epochs
        }

        # add epoch to each dataframe
        for epoch in rdicts:
            rdicts[epoch].df["epoch"] = epoch

        # if rdicts, then build dataframe, otherwise skip it.
        if rdicts:

            # concatenate all dataframes into one
            self.df = pd.concat([i.df for i in rdicts.values()])

            # sort values by year, and reset index without keeping old index
            self.df = (
                self.df
                .sort_values(by="year")
                .reset_index(drop=True)
                )
        else:
            self.df = pd.DataFrame([])

    @property
    def sdf(self):
        """
        Return a copy of the current .df dataframe selecting only the three
        most generally relevant columns: species, year, epoch, country,
        and stateProvince.
        This is only meant for viewing and will raise a warning if you try to
        modify it since it is a copy, and thus you would be setting values on
        a selection of a selection. See pandas docs in the warning for detalis.
        """
        return self.df[
            ["species", "year", "epoch", "country", "stateProvince"]]

    def simpsons_diversity(self, by):
        """
        Calculates simpon's diversity index: the probability that any two
        sampled individuals are the same species. Enter a key for groupby
        as a list of single or multiple keys.
        Parameters
        -----------
        by: str
            A column name used by .groupby to group samples prior to
            calculating simpson's diversity. For example, enter
        """
        # group on 'by' keyword, and exclude records missing data for 'by'.
        # and then count species in each group and calculate simp's div.
        data = (
            self.df[self.df[by].notna() & self.df.species.notna()]
            .groupby(by)
            .species
            .apply(calculate_simpsons_diversity)
            )

        # set zero values of simpson's diversity to nan
        data[data == 0] = np.nan
        return data


# utility functions
def load_epochs_from_csv(filepath):
    # init an empty epoch instance
    ep = Epochs("", 0, 0, 1)

    # load existing dataframe to instance's .df attribute
    ep.df = pd.read_csv(filepath, index_col=0)
    return ep


def calculate_simpsons_diversity(arr):
    "internal function to calculate simpson's diversity"
    simps = 0.
    for taxon in arr.unique():
        # proportion of individuals that are this species
        simps += (np.sum(arr == taxon) / arr.shape[0]) ** 2
    return 1. - simps


In [1]:
# Use the records library to download a series of occurrence records for Bombus
#import records

In [5]:
#rec = records.Records("Bombus", interval=(1990, 1994))
rec = Records("Bombus", interval=(1990, 1994))

In [6]:
rec.df.head()

,accessRights,associatedReferences,associatedTaxa,basisOfRecord,behavior,bibliographicCitation,catalogNumber,class,classKey,collectionCode,...,taxonKey,taxonRank,taxonRemarks,type,verbatimCoordinateSystem,verbatimElevation,verbatimEventDate,verbatimLocality,vernacularName,year
0,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,Insect Collection 259084,Insecta,216,Insect Collection,...,1340416,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993
1,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,Insect Collection 249962,Insecta,216,Insect Collection,...,1340472,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
2,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,Insect Collection 243363,Insecta,216,Insect Collection,...,1340409,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
3,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,Insect Collection 243311,Insecta,216,Insect Collection,...,1340409,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
4,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,Insect Collection 427057,Insecta,216,Insect Collection,...,1340350,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993


In [7]:
ep = Epochs("Bombus", 1990, 1994, 4)

In [9]:
ep.df.head()

,accessRights,associatedReferences,associatedTaxa,basisOfRecord,behavior,bibliographicCitation,catalogNumber,class,classKey,collectionCode,...,taxonRank,taxonRemarks,type,verbatimCoordinateSystem,verbatimElevation,verbatimEventDate,verbatimLocality,vernacularName,year,epoch
0,"Open Access, http://creativecommons.org/public...",NaN,NaN,PRESERVED_SPECIMEN,NaN,Bombus ternarius (YPM ENT 777699),YPM ENT 777699,Insecta,216,ENT,...,SPECIES,Animals and Plants: Invertebrates - Insects,PhysicalObject,NaN,NaN,NaN,NaN,bumble bees; apoid bees and wasps; bees wasps ...,1990,1990
1,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,JPS16112,Insecta,216,BBSL,...,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,1990
2,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,JPS16113,Insecta,216,BBSL,...,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,1990
3,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,JPS8532,Insecta,216,BBSL,...,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,1990
4,NaN,NaN,NaN,PRESERVED_SPECIMEN,NaN,NaN,BBSL225820,Insecta,216,BBSL,...,SPECIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,1990


In [11]:
ep.df.describe()

,classKey,coordinateUncertaintyInMeters,crawlId,day,decimalLatitude,decimalLongitude,elevation,elevationAccuracy,familyKey,genusKey,individualCount,key,kingdomKey,month,orderKey,phylumKey,speciesKey,taxonKey,year,epoch
count,3764.0,469.000000,3764.000000,3752.000000,3764.000000,3764.000000,306.000000,129.000000,3764.0,3764.0,1003.0,3.764000e+03,3764.0,3764.000000,3764.0,3764.0,3.594000e+03,3.764000e+03,3764.000000,3764.0
mean,216.0,60590.077079,83.902763,16.449360,40.651212,-96.087934,1850.913399,0.748062,4334.0,1340278.0,1.0,8.937197e+08,1.0,7.231403,1457.0,54.0,1.392087e+06,1.389747e+06,1992.547821,1990.0
std,0.0,37460.628947,161.615973,8.781485,5.726208,13.350641,944.183475,4.959651,0.0,0.0,0.0,3.100776e+08,0.0,1.435239,0.0,0.0,6.302663e+05,6.159591e+05,1.371901,0.0
min,216.0,0.250000,1.000000,1.000000,26.122300,-150.832320,60.000000,0.000000,4334.0,1340278.0,1.0,4.362873e+08,1.0,1.000000,1457.0,54.0,1.340280e+06,1.340278e+06,1990.000000,1990.0
25%,216.0,6461.160000,56.000000,9.000000,38.096240,-110.589749,1457.000000,0.000000,4334.0,1340278.0,1.0,6.994130e+08,1.0,7.000000,1457.0,54.0,1.340382e+06,1.340372e+06,1992.000000,1990.0
50%,216.0,90692.000000,56.000000,16.000000,42.895870,-92.899670,1798.000000,0.000000,4334.0,1340278.0,1.0,7.671389e+08,1.0,7.000000,1457.0,54.0,1.340416e+06,1.340416e+06,1993.000000,1990.0
75%,216.0,90692.000000,56.000000,24.000000,44.862500,-86.649375,2438.000000,0.000000,4334.0,1340278.0,1.0,9.115941e+08,1.0,8.000000,1457.0,54.0,1.340472e+06,1.340472e+06,1994.000000,1990.0
max,216.0,90692.000000,1032.000000,31.000000,61.427800,-67.880800,3901.000000,50.000000,4334.0,1340278.0,1.0,1.831505e+09,1.0,12.000000,1457.0,54.0,9.077942e+06,9.077942e+06,1994.000000,1990.0


## apply a machine learning method 

apply a machine learning method from the `scikit-learn` library to the data in the dataframe of your `records.Epochs` object. 

Select a machine learning class from scikit-learn. For this you can choose from many available options. Look to your reading for examples, or to the scikit learn documentation. The best way is to find examples of the model being applied and to substitute your data in for the example data.


*1. Select appropriate columns and format the data so that you have an column of labels (y) and one or more columns of features (X). Then split it into a training and test data set.

In [18]:
import toyplot

In [20]:
#elevation vs year; latitude vs elevation; day vs latitude, latitude vs year
# dataframes may be used just to view the data nicely
lat = ep.df.loc[:, 'decimalLatitude']
month = ep.df.loc[:, 'month']

data = pd.DataFrame({
    "X": month,
    "y": lat,
})
data.head()

,X,y
0,8,42.68952
1,3,33.22921
2,2,33.22921
3,7,47.04252
4,8,38.09624


In [22]:
# plot the data
toyplot.scatterplot(month, lat, height=250, width=300, size=3);

<g class="to

In [23]:
# split into training and testing data
# training size
tsize = 300

# convert to a 2d array
X = data.X.values[:, None]

# separate test from training
X_test = X[:tsize]
X_train = X[tsize:]

# show
print(X.shape)
print(X[:5])

(3764, 1)
[[8]
 [3]
 [2]
 [7]
 [8]]


In [24]:
# convert to a 1d array
y = data.y.values

# separate test from training
y_test = y[:tsize]
y_train = y[tsize:]

# show
print(y.shape)
print(y[:5])

(3764,)
[42.68952 33.22921 33.22921 47.04252 38.09624]


*2. Create an instance of that class.

In [25]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

*3. Train your model on your training data set (call .fit() with your model).

In [26]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

*4. Get predictions by applying your model to the test data set (call .predict() with your model).

In [27]:
yfit = model.predict(X_test)

*5. Measure the accuracy of your model by comparing the predicted values to the actual labels in your test data.

In [28]:
# compute r2 from comparing predicted y to actual y
from sklearn.metrics import r2_score, mean_squared_error

results ={
    "R2": r2_score(yfit, y_test),
    "MSE": mean_squared_error(yfit, y_test),
}
print(results)

{'R2': -101.50225563805229, 'MSE': 32.56129331477761}


In [30]:
pd.DataFrame({
    "Beta": [model.coef_[0]],
    "alpha": [model.intercept_],
    }, 
    index=["estimated"])


,Beta,alpha
estimated,-0.377901,43.471455


In [31]:
# build canvas
c = toyplot.Canvas(height=300, width=350)
a = c.cartesian()

# add training and test data points
a.scatterplot(X_train[:, 0], y_train, size=4, opacity=0.5);
a.scatterplot(X_test[:, 0], y_test, size=4, opacity=0.5);

# show that fitted line
a.plot(X_test[:, 0], yfit, color='black', style={"stroke-width": 2.5});

<g class="toy

*6. Describe the model that you tried to apply and the question that you tried to answer (e.g., I tried to use these features of the data to predict this). How well do you think the model worked?

I tried to use linear regression to answer the question: how does the month of the year affect the latitude at which bees are found? However, the relationship was not that linear and so the model fit was quite poor and predicted data did not fit the observed data well at all. The poor fit can be observed through the plot of observed vs predicted data. It can also been seen in the high mean squared error and R2 values. A better approach would be to determine the probability distribution at each month and predict based on that probability distribution rather than linear values.